# Simple Pandas Merges

The Pandas merge and join functions mirror the SQL merging functionality. The fundamental merge types are INNER, OUTER (or FULL OUTER,) LEFT and RIGHT. (I'm writing these in capital letters because SQL words are written that way.)


Let's begin with two dataframes: 'left' and 'right'

In [1]:
import pandas as pd

left  = pd.DataFrame({
    'some_match': [1, 'spam', 3, 6, 5],
    'none_match': [10, 20, 30, 40, 50],
    'all_match' : [5, 4, 3, 2, 1]
})

right = pd.DataFrame({
    'some_match': [1, 3, 2, 4, 5],
    'none_match': ['ten', 2.5, 'spam', 'eggs', 42],
    'all_match' : [5, 4, 3, 2, 1]
})

Note the column names - they describe how the data compares across the two data frames. Compare for yourself.

Now let's perform some merges. To begin, we'll try the inner merge:

![](images/A_inner_join_B.png)

We're always going to want the first two args to be "left" and "right". Plus we always want to give the kwarg "suffixes" so we know where columns come from.

To perform an inner join on the "some_match" column we'll want to pass those values to the "how" and "on" arguments as well.

In [2]:
pd.merge(left, right, how='inner', 
          on='some_match', suffixes=[
              '_left', '_right']
         )


,all_match_left,none_match_left,some_match,all_match_right,none_match_right
0,5,10,1,5,ten
1,3,30,3,4,2.5
2,1,50,5,1,42


## What happened here?

Note that our key - 'some_match' - is in the middle, and the DataFrame is only 3 rows long. The remaining two rows from the originals because, for example, left['some_match'] contains the value "spam" which does not appear in right['some_match']...

What if "some_match" looked a bit different? Let's change "spam" to the number 5. This means that the left dataframe will have two rows matching to one row in the right dataframe.

In [3]:
left['some_match'] = [1, 5, 3, 6, 5]

pd.merge(left, right, how='inner', 
          on='some_match', suffixes=[
              '_left', '_right']
         )


,all_match_left,none_match_left,some_match,all_match_right,none_match_right
0,5,10,1,5,ten
1,4,20,5,1,42
2,1,50,5,1,42
3,3,30,3,4,2.5


Now we have 4 rows, as expected. In the right dataframe there is only one row where the value is 5, in the left there are two - so we see duplicate values in the other columns which came over from the right dataframe.

You can try the code above with the "on" argument set to "all_match" or "none_match" just as a sanity check, so let's move on to full outer joins!

![](images/A_full_join_B.png)

The only change to our merge arguments is that we'll use "how=outer". This joins everything: if there's no match in the key column, Pandas will just make two rows, filling in the missing values with "Not a Number". Note the NaNs:

In [4]:
pd.merge(left, right, how='outer', 
          on='some_match', suffixes=[
              '_left', '_right']
         )


,all_match_left,none_match_left,some_match,all_match_right,none_match_right
0,5.0,10.0,1,5.0,ten
1,4.0,20.0,5,1.0,42
2,1.0,50.0,5,1.0,42
3,3.0,30.0,3,4.0,2.5
4,2.0,40.0,6,NaN,NaN
5,NaN,NaN,2,3.0,spam
6,NaN,NaN,4,2.0,eggs


Try this out with the "all_match" column and note the differences!

In [5]:
pd.merge(left, right, how='outer', 
          on='all_match', suffixes=[
              '_left', '_right']
         )

,all_match,none_match_left,some_match_left,none_match_right,some_match_right
0,5,10,1,ten,1
1,4,20,5,2.5,3
2,3,30,3,spam,2
3,2,40,6,eggs,4
4,1,50,5,42,5


Since everything matches, no NaNs!



## Now let's prioritize:

Until now, neither of the tables to be joined took priority over the other. Let's try a directional join now. Left and right joins take *everything* from the named side and only join data from the other side if there is a match.

![](images/A_left_join_B.png)

So, if we join on "none_match" we should only get NaNs from the opposite side side. On a left join, NaNs in the right side columns, for instance:

In [6]:
pd.merge(left, right, how='left', 
          on='none_match', suffixes=[
              '_left', '_right']
         )

,all_match_left,none_match,some_match_left,all_match_right,some_match_right
0,5,10,1,NaN,NaN
1,4,20,5,NaN,NaN
2,3,30,3,NaN,NaN
3,2,40,6,NaN,NaN
4,1,50,5,NaN,NaN


There are more joins out there, and I would encourage you to explore the Pandas merge documentation: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html

I hope you found this helpful, and please feel free to submit an issue if you see that I missed or misrepresented something.